## This demo app shows:
* How to use LangChain's YoutubeLoader to retrieve the caption in a YouTube video
* How to ask Llama to summarize the content (per the Llama's input size limit) of the video in a naive way using LangChain's stuff method
* How to bypass the limit of Llama's max input token size by using a more sophisticated way using LangChain's map_reduce and refine methods - see [here](https://python.langchain.com/docs/use_cases/summarization) for more info

We start by installing the necessary packages:
- [youtube-transcript-api](https://pypi.org/project/youtube-transcript-api/) API to get transcript/subtitles of a YouTube video
- [langchain](https://python.langchain.com/docs/get_started/introduction) provides necessary RAG tools for this demo
- [tiktoken](https://github.com/openai/tiktoken) BytePair Encoding tokenizer
- [pytube](https://pytube.io/en/latest/) Utility for downloading YouTube videos

**Note** This example uses Replicate to host the Llama model. If you have not set up/or used Replicate before, we suggest you take a look at the [HelloLlamaCloud](HelloLlamaCloud.ipynb) example for information on how to set up Replicate before continuing with this example.
If you do not want to use Replicate, you will need to make some changes to this notebook as you go along.

In [ ]:
!pip install langchain youtube-transcript-api tiktoken pytube

Let's load the YouTube video transcript using the YoutubeLoader.

In [1]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=1k37OcjH7BM", add_video_info=True
)

In [2]:
# load the youtube video caption into Documents
docs = loader.load()

In [3]:
# check the docs length and content
len(docs[0].page_content), docs[0].page_content[:300]

(26324,
 "so let's perhaps talk about each of these areas first deep learning that AI how the basic question how does a person interested in deep learning get started in the field the Atlanta AI is working to create courses to help people break into AI so my machine learning course that I taught through Stanf")

We are using Replicate in this example to host our Llama 2 model so you will need to get a Replicate token.

To get the Replicate token: 

- You will need to first sign in with Replicate with your github account
- Then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. 

**Note** After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

Alternatively, you can run Llama locally. See:
- [HelloLlamaCloud](HelloLlamaCloud.ipynb) for further information on how to run Llama using Replicate.
- [HelloLlamaLocal](HelloLlamaLocal.ipynb) for further information on how to run Llama locally.

In [4]:
# enter your Replicate API token, or you can use local Llama. See README for more info
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


 ········


Next we call the Llama 2 model from Replicate. In this example we will use the llama 2 13b chat model. You can find more Llama 2 models by searching for them on the [Replicate model explore page](https://replicate.com/explore?query=llama).

You can add them here in the format: model_name/version

If you using local Llama, just set llm accordingly - see the [HelloLlamaLocal notebook](HelloLlamaLocal.ipynb)

In [ ]:

from langchain.llms import Replicate

llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llm = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)

Once everything is set up, we prompt Llama 2 to summarize the first 4000 characters of the transcript for us.

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
prompt = ChatPromptTemplate.from_template(
    "Give me a summary of the text below: {text}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
# be careful of the input text length sent to LLM
text = docs[0].page_content[:4000]
summary = chain.run(text)
# this is the summary of the first 4000 characters of the video content
print(summary)

 Sure! Here's a summary of the text:

The speaker is discussing how to get started in deep learning, particularly for those who are self-taught. They recommend starting with the "Deep Learning Specialization" offered by Andrew Ng on Coursera, which covers everything from basic neural networks to advanced topics like attention models and sequence models. The prerequisites for the course are basic programming knowledge and high school math (specifically, understanding matrices and algebra).

The speaker notes that the course is designed to be accessible to both beginners and those with experience in machine learning, and that it


Next we try to summarize all the content of the transcript and we should get a `RuntimeError: Your input is too long. Max input length is 4096 tokens, but you supplied 5597 tokens.`.

In [7]:
# try to get a summary of the whole content
text = docs[0].page_content
summary = chain.run(text)
print(summary)


RuntimeError: Your input is too long. Max input length is 4096 tokens, but you supplied 5597 tokens.


Let's try some workarounds to see if we can summarize the entire transcript without running into the `RuntimeError`.

We will use the LangChain's `load_summarize_chain` and play around with the `chain_type`.


In [8]:
from langchain.chains.summarize import load_summarize_chain
# see https://python.langchain.com/docs/use_cases/summarization for more info
chain = load_summarize_chain(llm, chain_type="stuff") # other supported methods are map_reduce and refine
chain.run(docs)
# same RuntimeError: Your input is too long. but stuff works for shorter text with input length <= 4096 tokens

RuntimeError: Your input is too long. Max input length is 4096 tokens, but you supplied 5608 tokens.

In [9]:
chain = load_summarize_chain(llm, chain_type="refine")
# still get the "RuntimeError: Your input is too long. Max input length is 4096 tokens"
chain.run(docs)

RuntimeError: Your input is too long. Max input length is 4096 tokens, but you supplied 5608 tokens.


Since the transcript is bigger than the model can handle, we can split the transcript into chunks instead and use the [`refine`](https://python.langchain.com/docs/modules/chains/document/refine) `chain_type` to iteratively create an answer.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# we need to split the long input text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [11]:
# check the splitted docs lengths
len(split_docs), len(docs), len(split_docs[0].page_content), len(docs[0].page_content)

(2, 1, 15519, 26324)

In [12]:
# now get the summary of the whole docs - the whole youtube content
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(split_docs)

' Sure! Here is a revised version of the summary based on the additional information provided:\n\nAndrew Ng, co-founder of Coursera, discusses deep learning and AI. He emphasizes the importance of understanding the basics of machine learning before diving into deep learning. He also highlights the challenges students face when learning deep learning, such as struggling with concepts and overfitting. To overcome these challenges, he suggests breaking down complex topics into smaller, more manageable parts and using practical projects to build intuition. Additionally, he recommends a balanced approach to learning, including rein'

You can also use [`map_reduce`](https://python.langchain.com/docs/modules/chains/document/map_reduce) `chain_type` to implement a map reduce like architecture while summarizing the documents.

In [14]:
# another method is map_reduce
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(split_docs)

' Andrew Ng, an expert in deep learning and AI, offers advice for those interested in pursuing a career in this field. He stresses the importance of understanding the fundamentals of machine learning before delving into deep learning. He also highlights common challenges faced by students, such as grasping complex concepts and overfitting, and provides tips for overcoming these obstacles. Additionally, he emphasizes the value of developing a diverse skill set and building a strong network of peers and mentors.'

To investigate further, let's turn on Langchain's debug mode on to get an idea of how many calls are made to the model and the details of the inputs and outputs.
We will then run our summary using the `stuff` and `refine` `chain_types` and take a look at our output.

In [15]:
# to find how many calls to Llama have been made and the details of inputs and outputs of each call, set langchain to debug
import langchain
langchain.debug = True

# stuff method will cause the error in the end
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(split_docs)

[chain/start] [1:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:StuffDocumentsChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "text": "so let's perhaps talk about each of these areas first deep learning that AI how the basic question how does a person interested in deep learning get started in the field the Atlanta AI is working to create courses to help people break into AI so my machine learning course that I taught through Stanford means one of the most popular causes on Coursera to this day it's probably one of the courses sort of if I ask somebody how did you get into machine learning or how did you fall in love with machine learning or will get you interested it always goes back to rain and you rang at some point you were employing the amount of people you influence is Ruchika so for that I'm sure I speak for a lot of people say big thank you know yeah thank you you know I was once reading a news article I think it was tech 

RuntimeError: Your input is too long. Max input length is 4096 tokens, but you supplied 5610 tokens.

In [17]:
# but refine works
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(split_docs)

[chain/start] [1:chain:RefineDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RefineDocumentsChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "text": "so let's perhaps talk about each of these areas first deep learning that AI how the basic question how does a person interested in deep learning get started in the field the Atlanta AI is working to create courses to help people break into AI so my machine learning course that I taught through Stanford means one of the most popular causes on Coursera to this day it's probably one of the courses sort of if I ask somebody how did you get into machine learning or how did you fall in love with machine learning or will get you interested it always goes back to rain and you rang at some point you were employing the amount of people you influence is Ruchika so for that I'm sure I speak for a lot of people say big thank you know yeah thank you you know I was once reading a news article I think it was tec

' Sure! Here is a revised version of the summary based on the additional information provided:\n\nAndrew Ng, co-founder of Coursera, discusses deep learning and AI. He emphasizes the importance of understanding the basics of machine learning before diving into deep learning. He also highlights the challenges students face when learning deep learning, such as struggling with concepts and overfitting. To overcome these challenges, he suggests breaking down complex topics into smaller, more manageable parts and using practical projects to build intuition. Additionally, he recommends a diverse skill set and encourages learners'


As you can see, `stuff` fails because it tries to treat all the split documents as one and "stuffs" it into one prompt which leads to a much larger prompt than Llama 2 can handle while `refine` iteratively runs over the documents updating its answer as it goes.